---
## Docker外からコンテナ内のJupyter Notebookを使用する
#### 動機
　・機械学習のためにGPU環境を楽に整えたい  
　・（邪道だが）GUI的な使い方をしたい。しかしXを利用たリモート的な方法では上手くいかない  
  
#### 手法    
　・コンテナ内にJupyter notebookのサーバを立て、ブラウザ経由でアクセスする    
  
#### 前提
　・nvidia-dockerを利用する  
  
#### 実行した環境
　ハードウェア  
　　Core i7-7700CPU  
　　GeForce GTX 1080  
　OS  
　　ubuntu 16.04 LTS  
　docker  
　　nvidia-docker, Docker version 17.09.0-ce  
　　image : nvidia/cuda:8.0-cudnn5-devel
  　
#### 参考
[Docker Documentation](https://docs.docker.com/)    
[Dockerの使い方](https://qiita.com/tera_shin/items/efde65d2f875c5d8592a)by tera_shin  
[Dockerで作るJupyter Notebook開発環境](https://qiita.com/inu1one/items/075c372059370f6af909) by inu1one     
    

#### 行った手順
＜イメージ作成と動作確認＞  
　１．nvidia/cuda:8.0-cudnn5-develを継承し、必要なものをインストールするDockerfileを書いた   
　２．docker build 実行  
　３．作成したイメージからコンテナ立ち上げ  
　４．jupyter notebook --ip=0.0.0.0 --allow-root 実行  
　５．外部のブラウザから 0.0.0.0:指定したouter_ip/?token=<表示されたトークン> にアクセス　->　成功  
　６．jupyterから!nvidia-smi実行 -> 成功  
   
＜各種設定＞  
　７．作業用ユーザを設定  
　８．jupyter notebook --generate-config を実行してjupyter_notebook_config.py ファイルを生成    
　９．次のようにconfigファイルに追記      
  
```python
c.IPKernelApp.pylab = 'inline'
c.NotebookApp.ip = '*'
c.NotebookApp.open_browser = False
c.NotebookApp.port = <コンテナ内部での待受ポート>
c.NotebookApp.password = u'<パスワードのハッシュ値>'
```
   
　10．jupyter notebook 立ち上げをシェルスクリプトとして作成  
　11．docker commit してコンテナから設定済みイメージを作成  
　12．docker run から上のシェルスクリプト実行までを外部でシェルスクリプト化  
   
以上により手軽にGPUの使えるJupyter Notebookを立ち上げられるようになった。  

---
## Docker コマンド　簡易表
参考：  
　[Dockerコンテナの作成、起動〜停止まで](https://qiita.com/kooohei/items/0e788a2ce8c30f9dba53)  
  
メモ：  
　・コンテナIDまたはNAMEを指定する場合、対象を特定するのに十分な文字数を入力すれば実行できる（３文字でOK）  
　・Dockerfile内でapt-get updateしてからinstallする場合、コマンドを&&で繋いで一行で実行する必要がある  
　・Jupyter notebookのデフォルトipアドレスはlocalhostだが、Dockerfileはデフォルトを0.0.0.0にする関係で  
　　ipを振り直さないとjupyter立ち上げ時にエラーとなる。（[参考](https://github.com/kaczmarj/neurodocker/issues/82)）  
  　　
  
nvidia-docker導入はここら辺を  
　[Ubuntu 16.04 LTS で NVIDIA Docker を使ってみる](http://blog.amedama.jp/entry/2017/04/03/235901)    
    
 ===   
    
#### イメージの確認  

```
docker images
```

#### イメージの取得

```
docker pull <REPOSITORY>
```

#### Dockerfileからイメージの作成

```
docker build -t <IMAGE NAME>[:<TAG NAME>] <PATH to Dockerfile>
```

#### コンテナの稼働状況を確認

```
docker ps
docker ps -a # display all containers both running / stopped
```

####  コンテナの起動

```
docker start <CONTAINER ID or NAME>
```

#### コンテナの停止

```
docker stop <CONTAINER ID or NAME>
```

#### コンテナの削除

```
docker rm <CONTAINER ID or NAME>
```

#### 既にあるイメージから新しいイメージを作成する
・コンテナを起動する

```
docker run
 
```

・コンテナに接続する

```
docker attach
```

・イメージを作成する

```
docker commit
```

#### イメージの削除

```
docker rmi <IMAGE NAME:TAG>

#### Dockerfileの記述例
バージョン違いなどですぐに使えなくなるので、使用前に要確認  

```
FROM nvidia/cuda:8.0-cudnn5-devel

WORKDIR /workspace/
RUN apt-get update && apt-get install -y \
	build-essential libfreetype6-dev libpng12-dev libzmq3-dev pkg-config python3 python3-pip python3-dev python-virtualenv rsync software-properties-common curl git cmake swig python-opengl zip unzip wget sudo nano eog
RUN pip3 install --upgrade pip
RUN pip3 install --upgrade jupyter tensorflow-gpu keras matplotlib pandas sklearn Pillow h5py scipy

# bazel and sonnet
RUN apt-get install -y openjdk-8-jdk
RUN echo "deb [arch=amd64] http://storage.googleapis.com/bazel-apt stable jdk1.8" | tee /etc/apt/sources.list.d/bazel.list
RUN curl https://bazel.build/bazel-release.pub.gpg | apt-key add -
RUN apt-get update && apt-get install -y bazel
RUN apt-get upgrade -y bazel
RUN pip install --upgrade dm-sonnet-gpu

# openAI
RUN git clone https://github.com/openai/gym
RUN cd gym && pip install -e .[all]

# pytorch
RUN pip3 install http://download.pytorch.org/whl/cu80/torch-0.2.0.post3-cp35-cp35m-manylinux1_x86_64.whl 
RUN pip3 install torchvision

EXPOSE 80
```

---
## Ubuntu 16.04 で Caps Lock を Ctrl に置き換える
参考  
　[Ubuntu 16.04でcaps lockをCtrlに変更](http://soy-curd.hatenablog.com/entry/2016/07/09/105102)    
　[Ubuntu 16.04 CapsLockをControlに置換する方法](https://qiita.com/hirooooooaki/items/f404e76c6f171769412a)    
  
自分の環境では gnome-tweak-tool で解決  
　・run tweak tool  
　・"Typings" Tab -> Caps Lock Key behavior -> Check "Make Caps Lock an additional Ctrl"  
  